\
            # 53. 框架基础：FastAPI（FastAPI Fundamentals）

            目标：掌握 FastAPI 的核心能力：类型驱动的参数解析与校验、依赖注入、路由组织、返回模型、错误处理与测试。
本章示例对 FastAPI 做可选依赖处理；如果安装了 fastapi（及其依赖），可用 TestClient 在不启动端口的情况下运行示例。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- RESTful 接口设计基础
- Python 类型注解（建议了解）
- 异常处理


## 知识点地图

- 1. FastAPI 是什么：ASGI + 类型驱动开发
- 2. 最小应用：路径参数 + 查询参数 + Body（Pydantic）
- 3. 校验与错误：HTTPException、422、返回模型
- 4. 依赖注入 Depends：鉴权/资源管理的关键
- 5. 路由组织：APIRouter 与多模块项目结构（概念）
- 6. 运行与部署（概念）：uvicorn/gunicorn
- 7. 自动文档：OpenAPI/Swagger（理解即可）


## 自检清单（学完打勾）

- [ ] 会创建 FastAPI app 并定义路由（path/query/body）
- [ ] 理解 Pydantic 模型用于校验与序列化（请求/响应）
- [ ] 会写依赖注入（Depends）做鉴权/数据库连接管理（概念+示例）
- [ ] 理解状态码、异常（HTTPException）与统一错误格式
- [ ] 会用 TestClient 写接口测试（不需要起服务）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：FastAPI 是什么：ASGI + 类型驱动开发

- FastAPI 基于 ASGI：天然支持 async（I/O 并发更友好）。
- 强项：
  - 请求参数自动解析与校验（Pydantic）
  - 自动生成 OpenAPI/Swagger 文档
  - 依赖注入（Depends）组织横切逻辑（鉴权/DB session）

安装（建议）：
- `pip install fastapi uvicorn`


## 知识点 2：最小应用：路径参数 + 查询参数 + Body（Pydantic）

FastAPI 会根据类型注解自动完成：
- path 参数解析
- query 参数解析
- body 校验（Pydantic Model）

下面用 TestClient 直接调用，不需要起端口。


In [ ]:
try:
    from fastapi import FastAPI
    from pydantic import BaseModel
except Exception as e:
    print('fastapi not available:', type(e).__name__, e)
    print('install: pip install fastapi uvicorn')
else:
    try:
        from fastapi.testclient import TestClient
    except Exception as e:
        print('TestClient not available:', type(e).__name__, e)
        print('install full deps: pip install "fastapi[all]"')
    else:
        app = FastAPI()

        class ItemIn(BaseModel):
            title: str
            done: bool = False

        @app.get('/items/{item_id}')
        def get_item(item_id: int, q: str | None = None):
            return {'id': item_id, 'q': q}

        @app.post('/items', status_code=201)
        def create_item(item: ItemIn):
            return {'id': 1, **item.model_dump()}

        client = TestClient(app)
        print(client.get('/items/7?q=hi').status_code, client.get('/items/7?q=hi').json())
        print(client.post('/items', json={'title': 'learn fastapi'}).status_code, client.post('/items', json={'title': 'learn fastapi'}).json())


## 知识点 3：校验与错误：HTTPException、422、返回模型

- 请求 body 校验失败默认返回 422（Unprocessable Entity）并包含详细字段错误。
- 主动抛错用 `HTTPException(status_code=..., detail=...)`。
- 可以声明 `response_model` 控制输出字段（避免泄露敏感字段）。

实践：
- 统一错误格式（中大型项目常自定义 exception handler）。


## 知识点 4：依赖注入 Depends：鉴权/资源管理的关键

Depends 的价值：把“横切关注点”抽离出来：
- 鉴权：解析 token、校验权限
- DB session：每请求创建/关闭
- 限流/审计

下面示例演示一个最简 token 依赖。


In [ ]:
try:
    from fastapi import FastAPI, Depends, Header, HTTPException
except Exception as e:
    print('fastapi not available:', type(e).__name__, e)
else:
    try:
        from fastapi.testclient import TestClient
    except Exception as e:
        print('TestClient not available:', type(e).__name__, e)
    else:
        app = FastAPI()

        def require_token(x_token: str | None = Header(default=None)):
            if x_token != 'secret':
                raise HTTPException(status_code=401, detail='invalid token')
            return {'user': 'demo'}

        @app.get('/me')
        def me(user=Depends(require_token)):
            return user

        client = TestClient(app)
        print('no token ->', client.get('/me').status_code)
        print('with token ->', client.get('/me', headers={'X-Token': 'secret'}).status_code, client.get('/me', headers={'X-Token': 'secret'}).json())


## 知识点 5：路由组织：APIRouter 与多模块项目结构（概念）

- 用 `APIRouter` 拆分不同领域（users/orders/admin）。
- 主 app 里 include_router 并统一 prefix/tags。
- 配合依赖注入与配置管理，形成清晰可测试的结构。


## 知识点 6：运行与部署（概念）：uvicorn/gunicorn

本地运行：
- `uvicorn main:app --reload --port 8000`

生产：
- 常用 gunicorn + uvicorn worker（或直接 uvicorn 多进程）
- 需要反向代理、日志、健康检查、限流与监控


## 知识点 7：自动文档：OpenAPI/Swagger（理解即可）

FastAPI 默认提供：
- `/docs`（Swagger UI）
- `/redoc`
- `/openapi.json`

好处：
- 客户端可自动生成
- 更容易对齐接口契约


## 常见坑

- 把 async 当“自动更快”：CPU 重任务会阻塞事件循环
- 不设超时/不做重试：下游不稳定导致雪崩（与并发/网络章呼应）
- 直接返回 ORM 对象/内部模型：可能泄露敏感字段（要用 response_model）
- 依赖注入滥用：层级过深导致可读性差
- 不写测试：接口变更悄悄破坏客户端


## 综合小案例：用 FastAPI 写一个 Todo API（带校验与依赖注入）

要求：
- 使用 Pydantic Model 定义 TodoIn/TodoOut
- 路由：GET/POST/GET(id)/DELETE
- 依赖注入：用 Depends 注入一个“简单鉴权”或“存储层”
- 返回统一错误结构（可选：自定义 exception handler）


In [ ]:
# 建议：参考本章 TestClient 示例，扩展成完整 Todo API。
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- FastAPI 为什么适合写 API？它的“类型驱动”体现在哪里？
- 为什么校验失败默认是 422 而不是 400？（理解即可）
- Depends 解决了什么问题？常用于哪些场景？
- response_model 的作用是什么？


## 练习题（建议写代码）

- 实现 Todo API 并写 8 条 TestClient 测试（成功/失败/鉴权/404）。
- 把存储从内存换成 MongoDB 或 MySQL（与对应章节呼应）。
- 为接口生成 OpenAPI，并写一段“接口契约”说明（字段/状态码/错误结构）。
